### Add script to faq.xml

## Adding question into xml

In [14]:
# 2 
# Mar6_set1 copy.json
import json
from pprint import pprint

with open('Mar6_set1 copy.json') as f:
    data = json.load(f)

pprint(data)

{'dyn_faqs': {'Apply': ['可唔可以電話apply{@Nickname 定期存款}'],
              'Eligibility': ['我每月薪金一萬七 可以申請{@Nickname money back 信用卡}嗎？',
                              '需要什麼證明文件',
                              '最低存款額係幾多',
                              '最低存款額',
                              '最低存款？'],
              'GetStatement_DynFAQ': ['我想問我credit card {@GetStatementType '
                                      'statement}',
                                      '我想睇我{@Nickname 信用卡}嘅{@GetStatementType '
                                      '網上月結單}',
                                      '我成功登入{@Nickname '
                                      '網上理財}後，冇法查閱我最近{@Nickname 信用卡}的消費紀錄'],
              'Promotions_DynFAQ': ['海外優惠呢？',
                                    '想問下最近{@Nickname asiamiles '
                                    'mastercard}去海外簽帳有冇promotion?'],
              'WelcomeOffers': ['現在申請這張有什麼迎新禮品？',
                                '請問{@Nickname simply card} 迎新優惠有＄1111回贈的詳情']},
 'faqs'

In [15]:

TAIL = 4

# --------------
from xml.etree.ElementTree import ElementTree,Element
 
def read_xml(in_path):
    '''读取并解析xml文件
       in_path: xml路径
       return: ElementTree'''
    tree = ElementTree()
    tree.parse(in_path)
    return tree
 
def write_xml(tree, out_path):
    '''将xml文件写出
       tree: xml树
       out_path: 写出路径'''
    tree.write(out_path, encoding="utf-8",xml_declaration=True)
 
    
def if_match(node, kv_map):
    '''判断某个节点是否包含所有传入参数属性
       node: 节点
       kv_map: 属性及属性值组成的map'''
    for key in kv_map:
        if node.get(key) != kv_map.get(key):
            return False
    return True
 
#---------------search -----
 
def find_nodes(tree, path):
    '''查找某个路径匹配的所有节点
       tree: xml树
       path: 节点路径'''
    return tree.findall(path)
 
 
def get_node_by_keyvalue(nodelist, kv_map):
    '''根据属性及属性值定位符合的节点，返回节点
       nodelist: 节点列表
       kv_map: 匹配属性及属性值map'''
    result_nodes = []
    for node in nodelist:
        if if_match(node, kv_map):
            result_nodes.append(node)
    return result_nodes


def get_node_by_tag(nodelist, tag):
    '''根据tag定位符合的节点，返回节点
       nodelist: 节点列表
       tag 节点类型'''
    result_nodes = []
    for node in nodelist:
        if node.tag == tag:
            result_nodes.append(node)
    return result_nodes
 
#---------------change -----
 
def change_node_properties(nodelist, kv_map, is_delete=False):
    '''修改/增加 /删除 节点的属性及属性值
       nodelist: 节点列表
       kv_map:属性及属性值map'''
    for node in nodelist:
        for key in kv_map:
            if is_delete: 
                if key in node.attrib:
                    del node.attrib[key]
            else:
                node.set(key, kv_map.get(key))
            
def change_node_text(nodelist, text, is_add=False, is_delete=False):
    '''改变/增加/删除一个节点的文本
       nodelist:节点列表
       text : 更新后的文本'''
    for node in nodelist:
        if is_add:
            node.text += text
        elif is_delete:
            node.text = ""
        else:
            node.text = text
            
def create_node(tag, property_map, content, tail=TAIL):
    '''新造一个节点
       tag:节点标签
       property_map:属性及属性值map
       content: 节点闭合标签里的文本内容
       return 新节点'''
    element = Element(tag, property_map)
    element.text = content
    element.tail = "\n" + tail*"  "
    return element
        
def add_child_node(nodelist, element, tail=TAIL):
    '''给一个节点添加子节点
       nodelist: 节点列表
       element: 子节点'''
    # 调节indent
    if len(nodelist[-1]) > 0:
        last_node = nodelist[-1][-1]
        last_node.tail = "\n" + tail*"  "
    for node in nodelist:
        node.append(element)
        
def del_node_by_tagkeyvalue(nodelist, tag, kv_map):
    '''同过属性及属性值定位一个节点，并删除之
       nodelist: 父节点列表
       tag:子节点标签
       kv_map: 属性及属性值列表'''
    for parent_node in nodelist:
        children = parent_node.getchildren()
        for child in children:
            if child.tag == tag and if_match(child, kv_map):
                parent_node.remove(child)
                        
 

In [16]:
import hashlib

def hashify(text):
    m = hashlib.md5()
    m.update(text.encode('utf-8'))
    md5_id = m.hexdigest()
    return md5_id


In [17]:
def find_nodes(tree, path):
    '''查找某个路径匹配的所有节点
       tree: xml树
       path: 节点路径'''
    return tree.findall(path)


def get_all_by_scid(scid, path):
    print(scid)
    tree = read_xml(path)
    
    nodes = find_nodes(tree, "intents/intent/qa")
    
    #scid = "SC86"
    scid = "VpaGetAnswer__Question--String__"+scid
    
    result_nodes = get_node_by_keyvalue(nodes, {"id": scid})
    
    res_nodes = get_node_by_tag(result_nodes[0], tag="question")
    
    lst = list()
    
    for node in res_nodes:
        
        lst.append(node.text)
    
    return lst


def insert_xml(toAddList, scid, path):
    # --------------faq------------
    tree = read_xml(path)
    
    nodes = find_nodes(tree, "intents/intent/qa")
    
    result_nodes = get_node_by_keyvalue(nodes, {"id": scid})
    
    #--------------func------------------------
    def faq_adding(seeds, path):
        
        for i in range(len(seeds)):
            content = seeds[i]
            
            a = create_node("question", {"id":hashify(content),"scope":"train", "source":"seed"}, content)
            if i==len(seeds)-1:
                a = create_node("question", {"id":hashify(content),"scope":"train", "source":"seed"}, content, tail=3)
                
            add_child_node(result_nodes, a)
    
        write_xml(tree, path)
    #-------------------------------------
    
    faq_adding(toAddList, path)
    
    print(scid + "finished.")



In [18]:
# --------------faq------------
path = 'faq_2_added.xml'
tree = read_xml(path)

nodes = find_nodes(tree, "intents/intent/qa")

for key in data: 
    r = data[key]
    
    for a, b in r.items():
        scid = a
        toAddList = list(b)
        result_nodes = get_node_by_keyvalue(nodes, {"label": scid})    
    
        if len(result_nodes) == 0:
            print (scid)
            continue
    
        for i in range(len(toAddList)):
            content = toAddList[i]

            a = create_node("question", {"id":hashify(content),"scope":"train.annotation", "source":"scb.annotation.190226"}, content)
            if i==len(toAddList)-1:
                a = create_node("question", {"id":hashify(content),"scope":"train.annotation", "source":"scb.annotation.190226"}, content, tail=3)
            add_child_node(result_nodes, a)


new_path = "faq_3_added.xml"
write_xml(tree, new_path)

SC423
SC503
ATMFinder


In [13]:
#2
#data.keys()
#data['dyn_faqs']
#data[data.keys()]

for key in data: 
    r = data[key]
    print (r)
    #print (key)



{'Eligibility': ['我每月薪金一萬七 可以申請{@Nickname money back 信用卡}嗎？', '需要什麼證明文件', '最低存款額係幾多', '最低存款額', '最低存款？'], 'GetStatement_DynFAQ': ['我想問我credit card {@GetStatementType statement}', '我想睇我{@Nickname 信用卡}嘅{@GetStatementType 網上月結單}', '我成功登入{@Nickname 網上理財}後，冇法查閱我最近{@Nickname 信用卡}的消費紀錄'], 'Apply': ['可唔可以電話apply{@Nickname 定期存款}'], 'WelcomeOffers': ['現在申請這張有什麼迎新禮品？', '請問{@Nickname simply card} 迎新優惠有＄1111回贈的詳情'], 'Promotions_DynFAQ': ['海外優惠呢？', '想問下最近{@Nickname asiamiles mastercard}去海外簽帳有冇promotion?']}
{'G1011': ['稅務居留國之確認書下載'], 'G1015': ['怎麼開公司戶口', '預約公司開戶'], 'G374': ['八達通自動增值服務'], 'G462': ['電子帳戶'], 'G400': ['萬里通萬事達信用卡'], 'G444': ['請問如何申請中小企戶口?'], 'G414': ['用來出糧', '出糧戶口推廣', '有關開立渣打出糧戶口事宜', '出糧戶口回贈'], 'G1010': ['想補交申請信用卡文件'], 'G498': ['我想致電bank confirmation department', '渣打銀行的電話', '電話號碼', '不知道聯係電話是什麼？', '你有冇家居保險嘅熱線電話', '有冇電話可以直接問'], 'G451': ['亞洲萬里通'], 'G463': ['網上估價', '我想做網上物業估價'], 'G377': ['要收費?'], 'G80': ['想問有關按揭事宜'], 'G449': ['請問如何登記為「渣打亞洲萬里通萬事達卡會員推薦計劃 」的推薦人?'], 'G428': ['111萬定期', '定期'], 'G389

In [ ]:
#2
